In [241]:
import pandas as pd
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
import ollama
from langchain.llms import Ollama
from langchain.chains import LLMChain
import json
import requests
from langchain.llms import BaseLLM

In [242]:
fewShotData = pd.read_csv('../Training Dataset/FewShotForEvents.csv')
fewShotData = fewShotData[['content', 'events refined']]
fewShotData = fewShotData.dropna()
fewShotData.head()

,content,events refined
0,Keeps logging me out of account saying session...,"[It keeps logging me out, It is saying session..."
1,Facebook marketplace has a bug or a terrible f...,"[I search and scroll, It will select a random ..."
2,I don't know what has happened lately it will ...,"[I log in with the right password, The app sta..."
3,Can't login at all can't recover my account at...,"[I have tried uninstalling, I tried to get the..."
4,When I try to share things that I like on my w...,"[I try to share things, it throws it in the ir..."


In [ ]:
few_shot_examples = fewShotData.to_dict(orient='records')

examplePrompt = PromptTemplate(
    input_variables=['content', 'events refined'],
    template="Reviews: {content}\nEvents: {events refined}\n"
)

fewShotPrompt = FewShotPromptTemplate(
    examples=few_shot_examples,
    example_prompt=examplePrompt,
    suffix="Reviews: {content}",
    input_variables=['content']
)

formatted_prompt = fewShotPrompt.format(content="I went to the store today.")
print(formatted_prompt)

Reviews: Keeps logging me out of account saying session expired and everytime I try to log in it won't let me, keeps updating constantly even when I keep it up to date so not sure if the app is glitching, overall not a good app so may have to consider deleting Facebook as this has put me off and happens way too much, keep trying to update the app thinking it was that but its still not working and it says update again even after i already did
Events: [It keeps logging me out, It is saying session expired, I try to log in, It won't let me log in][I keep trying to update the app, It still won't let me log in][I update app, It says update again]


Reviews: Facebook marketplace has a bug or a terrible feature where it will select a random picture as the main one (so the only one buyers see when searching and scrolling) keeps selecting a zoomed up tiny area of damage as the main photo instead of the actual overall photo of an object no wonder no one but scam bots message me now.
Events: [I s

In [ ]:
ollama.list()

{'models': [{'name': 'fewShotLLAMA2:latest',
   'model': 'fewShotLLAMA2:latest',
   'modified_at': '2024-06-01T03:07:08.2162239+06:00',
   'size': 3826794200,
   'digest': '1e28f7db7135fece538a276a7fad66290310b6db7383a44bd0a9065a573191de',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '6.7B',
    'quantization_level': 'Q4_0'},
   'expires_at': '0001-01-01T00:00:00Z'},
  {'name': 'EventExtraction-llama2:latest',
   'model': 'EventExtraction-llama2:latest',
   'modified_at': '2024-05-31T02:42:47.3735996+06:00',
   'size': 3826794719,
   'digest': '4c7872e5e274c752b29d74d425a228af534ee19be33ea4c149da09ad4add9d80',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '6.7B',
    'quantization_level': 'Q4_0'},
   'expires_at': '0001-01-01T00:00:00Z'},
  {'name': 'llama2:latest',
   'model': 'llama2:latest',
   'modified_at': '2024-05-

In [ ]:
class Ollama(BaseLLM):
    base_url: str
    model: str

    def _call(self, prompt, stop=None):
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",  # Adjust the endpoint if necessary
                json={"model": self.model, "prompt": prompt}
            )
            response.raise_for_status()  # Raise an error for bad HTTP status codes
            raw_content = response.content.decode('utf-8')
            print("Raw Response content:", raw_content)  # Debugging
            
            # Handle multiple JSON objects
            response_json = None
            for part in raw_content.split('\n'):
                try:
                    response_json = json.loads(part)
                    break  # Stop after the first valid JSON object
                except json.JSONDecodeError:
                    continue
            
            if response_json:
                print("Response JSON:", response_json)  # Debugging
                return response_json.get('text', '')
            else:
                print("No valid JSON found in response.")
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
        except ValueError as e:
            print(f"JSON decode error: {e}")
            print(f"Response content: {raw_content}")
        return ""

    def _generate(self, prompts, stop=None):
        return [self._call(prompt, stop) for prompt in prompts]

    @property
    def _llm_type(self):
        return "ollama"

    @property
    def _identifying_params(self):
        return {"base_url": self.base_url, "model": self.model}

In [244]:
ollama = Ollama(base_url='http://localhost:11434', model='fewShotLLAMA2')

In [239]:
llamaChain = LLMChain(llm=ollama, prompt=fewShotPrompt)

In [240]:
chain_response = llamaChain.run("I went to the store today.")
print(chain_response)

Raw Response content: {"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.1005823Z","response":"It","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.1170628Z","response":" seems","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.1349074Z","response":" like","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.1520034Z","response":" you","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.1700198Z","response":" are","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.1857243Z","response":" collect","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.2031818Z","response":"ing","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.2199175Z","response":" reviews","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.2368575Z","response":" from","done":false}
{"model":"fewShotLLAMA2","created_at":"2024-06-01T11:59:08.2

AttributeError: 'list' object has no attribute 'flatten'